In [6]:
import re
import requests
from bs4 import BeautifulSoup

def morningstar(url):

#    url = "http://www.morningstar.se/Funds/Quicktake/Overview.aspx?perfid=0P0000J24W"

    r  = requests.get(url)

    data = r.text

    soup = BeautifulSoup(data)
    a = soup.find('table', {"class": "alternatedtoplist"}).findAll('td')[1].contents[0]
    
    # todo: should care about currency

    a2 = map(int, re.findall(r'\d+', a))
    a3 = float('{}.{}'.format(a2[0], a2[1]))
    return a3

#morningstar("http://www.morningstar.se/Funds/Quicktake/Overview.aspx?perfid=0P0000J24W")

In [7]:
def yahoo_share(share_name):
    from yahoo_finance import Share, Currency
    yahoo = Share(share_name)
    price = float(yahoo.get_price())
    
    stock_currency = yahoo.data_set['Currency']
    
    if stock_currency != 'SEK':
        currency_convert = Currency('{}SEK'.format(stock_currency))
        rate = currency_convert.get_rate()
        price_in_SEK = float(price) * float(rate)
    else:
        price_in_SEK = float(price)
    return price_in_SEK

#print yahoo_share('AAPL')
#print yahoo_share('VITR.ST')

In [8]:
assets = {
    'AAPL': {
        'own': 1,
        'current_value': yahoo_share('AAPL')
    },
    'Coca-Cola': {
        'own': 3,
        'current_value': yahoo_share('KO')
    },
    'Duke Energy': {
        'own': 2,
        'current_value': yahoo_share('DUK')
    },
    'NOF': {
        'own': 170,
        'current_value': yahoo_share('NOF.OL')
    },
    'Target': {
        'own': 2,
        'current_value': yahoo_share('TGT')
    },
    'VITR': {
        'own': 10,
        'current_value': yahoo_share('VITR.ST')
    },
    
    'Carnegie Indien': {
        'own': 4.2735,
        'current_value': morningstar("http://www.morningstar.se/Funds/Quicktake/Overview.aspx?perfid=0P00000ZOO")
    },
    'JPM Global Health': {
        'own': 2.11,
        'current_value': morningstar("http://www.morningstar.se/Funds/Quicktake/Overview.aspx?perfid=0P0000LZRB")
    },
    'Nordnet Superfonden Sverige': {
        'own': 6.4099,
        'current_value': morningstar("http://www.morningstar.se/Funds/Quicktake/Overview.aspx?perfid=0P0000J24W")
    },
    'SEB Japanfond': {
        'own': 491.8263,
        'current_value': morningstar("http://www.morningstar.se/Funds/Quicktake/Overview.aspx?perfid=0P00000LU7")
    },    
}

In [9]:
from ipy_table import *
def calculate_total_assets():
    total_worth = 0
    table = [
        ['Asset', 'Own', 'Price', 'Worth']
    ]
    for asset_name in assets:
        worth = assets[asset_name]['current_value']*assets[asset_name]['own']
        total_worth += worth
        table.append([asset_name, assets[asset_name]['own'], assets[asset_name]['current_value'], worth])
        #print '{}\t{}\t{}'.format(asset_name, assets[asset_name]['current_value'], worth)
    table.append(['', '', '', ''])
    table.append(['', '', '', total_worth])
    #print(total_worth)
    return table

make_table(calculate_total_assets())
apply_theme('basic')

Asset,Own,Price,Worth
Duke Energy,2,563.0259,1126.0518
Target,2,641.6222,1283.2443
VITR,10,163.0000,1630.0000
SEB Japanfond,491.8263,6.5500,3221.4623
NOF,170,7.6285,1296.8484
AAPL,1,940.9305,940.9305
Nordnet Superfonden Sverige,6.4099,280.5200,1798.1051
Carnegie Indien,4.2735,368.6500,1575.4258
Coca-Cola,3,314.1378,942.4135
JPM Global Health,2.1100,303.1000,639.5410
